In [ ]:
!pip install requests beautifulsoup4 pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import numpy as np

# **Read Files**

In [ ]:
zhvi = pd.read_csv("raw_data/ZillowHomeValueIndex.csv")
zori = pd.read_csv("raw_data/ZillowObservedRentIndex.csv")
zori1 = pd.read_csv("raw_data/ZillowObservedRentIndex1.csv")
crime_2024 = pd.read_excel("raw_data/NIBRSPublicView2024.xlsx")
crime_2025 = pd.read_excel("raw_data/NIBRSPublicView2025.xlsx")
property_listings = pd.read_csv("raw_data/PropertyListings.csv")

# **1. Preprocess Data**

## **1.1 Property Listings Data**

In [ ]:
property_listings.head(2)

,zip,price,address,beds,baths,sqft
0,77002,"$509,000","120 Pierce St, Houston, TX 77002",3 beds,3.5 baths,"2,015"
1,77002,"$175,000","2016 Main St #704, Houston, TX 77002",2 beds,2 baths,"1,169"


In [ ]:
property_listings

,zip,price,address,beds,baths,sqft
0,77002,"$509,000","120 Pierce St, Houston, TX 77002",3 beds,3.5 baths,"2,015"
1,77002,"$175,000","2016 Main St #704, Houston, TX 77002",2 beds,2 baths,"1,169"
2,77002,"$160,000","2016 Main St Unit 105-107, Houston, TX 77002",0 beds,— baths,710
3,77002,"$250,000","2016 Main St #1502, Houston, TX 77002",2 beds,2 baths,"1,145"
4,77002,"$250,000","2016 Main St #1809, Houston, TX 77002",2 beds,2 baths,908
...,...,...,...,...,...,...
12462,77099,"$340,000","12318 Bexley Dr, Houston, TX 77099",4 beds,2 baths,"1,896"
12463,77099,"$330,000","12314 Bexley Dr, Houston, TX 77099",3 beds,2 baths,"1,642"
12464,77099,"$1,400,000","9310 Cook Rd, Houston, TX 77099",— beds,— baths,—
12465,77099,"$700,000","0 Kinghurst Dr, Houston, TX 77099",— beds,— baths,—


### **Remove missing rows**

In [ ]:
property_listings = property_listings.dropna()
property_listings

,zip,price,address,beds,baths,sqft
0,77002,"$509,000","120 Pierce St, Houston, TX 77002",3 beds,3.5 baths,"2,015"
1,77002,"$175,000","2016 Main St #704, Houston, TX 77002",2 beds,2 baths,"1,169"
2,77002,"$160,000","2016 Main St Unit 105-107, Houston, TX 77002",0 beds,— baths,710
3,77002,"$250,000","2016 Main St #1502, Houston, TX 77002",2 beds,2 baths,"1,145"
4,77002,"$250,000","2016 Main St #1809, Houston, TX 77002",2 beds,2 baths,908
...,...,...,...,...,...,...
12462,77099,"$340,000","12318 Bexley Dr, Houston, TX 77099",4 beds,2 baths,"1,896"
12463,77099,"$330,000","12314 Bexley Dr, Houston, TX 77099",3 beds,2 baths,"1,642"
12464,77099,"$1,400,000","9310 Cook Rd, Houston, TX 77099",— beds,— baths,—
12465,77099,"$700,000","0 Kinghurst Dr, Houston, TX 77099",— beds,— baths,—


### **Convert Price field into numeric**

In [ ]:
property_listings.dtypes

,0
zip,int64
price,object
address,object
beds,object
baths,object
sqft,object


In [ ]:
property_listings['price'] = (
    property_listings['price']
    .replace('[\$,]','', regex=True)
    .str.strip()
    .replace('', pd.NA)
)

property_listings['price'] = pd.to_numeric(property_listings['price'], errors='coerce')

property_listings.dropna(subset=['price'], inplace=True)

### **Convert zipcode to string**

In [ ]:
property_listings['zip'] = property_listings['zip'].astype('str')
property_listings.rename(columns={'zip':'zipcode'}, inplace=True)

### **Convert Beds field into numeric**

In [ ]:
property_listings['beds']=property_listings['beds'].str.extract(r'(\d+)')
property_listings.dropna(subset=['beds'],inplace=True)
# property_listings['beds'] = property_listings['beds'].astype(int)

### **Convert Baths field to numeric**

In [ ]:
property_listings['baths']=property_listings['baths'].str.extract(r'(\d+\.?\d*)')
property_listings.dropna(subset=['baths'],inplace=True)

### **Convert Square Footage to Numeric**

In [ ]:
property_listings = property_listings[property_listings['sqft'].str.replace(',', '').str.isnumeric()]
property_listings['sqft'] = property_listings['sqft'].replace(',','',regex=True).astype(float)

<ipython-input-12-6658f79375d6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_listings['sqft'] = property_listings['sqft'].replace(',','',regex=True).astype(float)


### **Address into multiple fields**

In [ ]:
property_listings['city']=property_listings['address'].apply(lambda x: x.split(',')[-2].strip())
property_listings['state']=property_listings['address'].apply(lambda x: x.split(',')[-1].split()[0].strip())

<ipython-input-13-feeea06289dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_listings['city']=property_listings['address'].apply(lambda x: x.split(',')[-2].strip())
<ipython-input-13-feeea06289dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_listings['state']=property_listings['address'].apply(lambda x: x.split(',')[-1].split()[0].strip())


### **Select only city= 'Houston' rows**

In [ ]:
property_listings = property_listings[property_listings['city'] == 'Houston']

In [ ]:
property_listings = property_listings.apply(lambda col: col.str.lower() if col.dtype == "object" else col)
property_listings

,zipcode,price,address,beds,baths,sqft,city,state
0,77002,509000.0,"120 pierce st, houston, tx 77002",3,3.5,2015.0,houston,tx
1,77002,175000.0,"2016 main st #704, houston, tx 77002",2,2,1169.0,houston,tx
3,77002,250000.0,"2016 main st #1502, houston, tx 77002",2,2,1145.0,houston,tx
4,77002,250000.0,"2016 main st #1809, houston, tx 77002",2,2,908.0,houston,tx
5,77002,225000.0,"300 st joseph pkwy #203, houston, tx 77002",1,1,1015.0,houston,tx
...,...,...,...,...,...,...,...,...
12459,77099,108000.0,"9797 leawood blvd #1411, houston, tx 77099",2,2,1068.0,houston,tx
12460,77099,320000.0,"12510 shannon hills dr, houston, tx 77099",3,2,1642.0,houston,tx
12461,77099,330000.0,"12326 bexley dr, houston, tx 77099",3,2,1642.0,houston,tx
12462,77099,340000.0,"12318 bexley dr, houston, tx 77099",4,2,1896.0,houston,tx


## **1.2 Zillow Home Value Index Data**


In [ ]:
zhvi.dtypes

,0
RegionID,int64
SizeRank,int64
RegionName,int64
RegionType,object
StateName,object
...,...
2024-10-31,float64
2024-11-30,float64
2024-12-31,float64
2025-01-31,float64


In [ ]:
zhvi.head(2)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,209414.041482,...,495936.411427,496241.930564,495863.854951,496270.138911,496959.078506,498079.880762,498797.284325,499214.713899,499637.493567,499316.860950
1,61148,2,8701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,131918.577791,...,584298.024483,589505.709784,594184.490210,599212.219688,605133.048238,610430.184997,613806.807609,615811.473737,616448.543879,618664.793443


### **Replace NA values with 0**

In [ ]:
zhvi.fillna(0,inplace=True)

### **Verify Consistency Between 'StateName' and 'State'**


In [ ]:
(zhvi['StateName'] == zhvi['State']).all()

np.True_

### **Remove repeated or unecessary columns**

In [ ]:
zhvi = zhvi.drop(columns=['RegionType','StateName'])

### **Rename columns**

In [ ]:
zhvi.rename(columns={'RegionID':'region_id','SizeRank':'size_rank','RegionName':'zipcode','CountyName':'county'},inplace=True)

### **Reshape ZHVI Data to Long Format and Parse Dates**


In [ ]:
id_vars = ['region_id','size_rank','zipcode','county','State','City','Metro']

zhvi = zhvi.melt(id_vars = id_vars, var_name = "date", value_name = "zhvi_value")

zhvi['date'] = pd.to_datetime(zhvi['date'])

### **Convert zipcode to str**

In [ ]:
zhvi['zipcode'] = zhvi['zipcode'].astype('str')

In [ ]:
zhvi.dtypes

,0
region_id,int64
size_rank,int64
zipcode,object
county,object
State,object
City,object
Metro,object
date,datetime64[ns]
zhvi_value,float64


### **Standardize Column Names and Lowercase String Columns**

In [ ]:
zhvi = zhvi.apply(lambda col: col.str.lower() if col.dtype == "object" else col)
zhvi.columns = zhvi.columns.str.lower()
zhvi.head(2)

,region_id,size_rank,zipcode,county,state,city,metro,date,zhvi_value
0,91982,1,77494,fort bend county,tx,katy,"houston-the woodlands-sugar land, tx",2000-01-31,209414.041482
1,61148,2,8701,ocean county,nj,lakewood,"new york-newark-jersey city, ny-nj-pa",2000-01-31,131918.577791


## **1.3 Zillow Observed Rent Index Data**

In [ ]:
zori.head(1)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1523.412518,...,1929.997123,1932.100456,1928.31139,1933.758912,1932.781268,1932.649706,1942.156741,1948.045995,1951.49988,1944.607081


### **Replace NA values with 0**

In [ ]:
zori.fillna(0, inplace=True)
zori1.fillna(0, inplace=True)

### **Verify Consistency Between 'StateName' and 'State'**

In [ ]:
(zori['StateName'] == zori['State']).all()
(zori1['StateName'] == zori1['State']).all()

np.True_

### **Remove repeated or unecessary columns**

In [ ]:
zori = zori.drop(columns=['RegionType','StateName'])
zori1 = zori1.drop(columns=['RegionType','StateName'])

### **Rename columns**

In [ ]:
zori.rename(columns={'RegionID':'region_id','SizeRank':'size_rank','RegionName':'zipcode','CountyName':'county'},inplace=True)
zori1.rename(columns={'RegionID':'region_id','SizeRank':'size_rank','RegionName':'zipcode','CountyName':'county'},inplace=True)

### **Reshape ZHVI Data to Long Format and Parse Dates**


In [ ]:
id_vars = ['region_id','size_rank','zipcode','county','State','City','Metro']

zori = zori.melt(id_vars = id_vars, var_name = "date", value_name = "zori_value")

zori['date'] = pd.to_datetime(zori['date'])

zori1 = zori1.melt(id_vars = id_vars, var_name = "date", value_name = "zori_value")

zori1['date'] = pd.to_datetime(zori1['date'])

### **Convert zipcode to str**

In [ ]:
zori['zipcode'] = zori['zipcode'].astype('str')
zori1['zipcode'] = zori1['zipcode'].astype('str')

In [ ]:
zori.dtypes

,0
region_id,int64
size_rank,int64
zipcode,object
county,object
State,object
City,object
Metro,object
date,datetime64[ns]
zori_value,float64


### **Standardize Column Names and Lowercase String Columns**

In [ ]:
zori = zori.apply(lambda col: col.str.lower() if col.dtype == "object" else col)
zori.columns = zori.columns.str.lower()

zori1 = zori1.apply(lambda col: col.str.lower() if col.dtype == "object" else col)
zori1.columns = zori1.columns.str.lower()

## **1.3 Crime Data 2024**

In [ ]:
crime_2024.head()

,Incident,RMSOccurrenceDate,RMSOccurrenceHour,NIBRSClass,NIBRSDescription,OffenseCount,Beat,Premise,StreetNo,StreetName,StreetType,Suffix,City,ZIPCode,MapLongitude,MapLatitude
0,12124,2024-01-01,0,90C,Disorderly conduct,1,8C30,"Residence, Home (Includes Apartment)",8404,EASTOVER,ST,NaN,HOUSTON,77028.0,-95.271852,29.823953
1,14424,2024-01-01,0,90C,Disorderly conduct,1,8C10,"Residence, Home (Includes Apartment)",7125,GLASS,ST,NaN,HOUSTON,77016.0,-95.325797,29.824464
2,15924,2024-01-01,0,13A,Aggravated Assault,1,6B10,"Residence, Home (Includes Apartment)",329,INNSDALE,DR,NaN,HOUSTON,77076.0,-95.371408,29.850945
3,17024,2024-01-01,0,520,Weapon law violations,1,16E20,"Residence, Home (Includes Apartment)",3727,DALMATIAN,DR,NaN,HOUSTON,77045.0,-95.432224,29.618177
4,17624,2024-01-01,0,13A,Aggravated Assault,1,1A20,"Parking Lot, Garage",230,ALABAMA,ST,W,HOUSTON,77006.0,-95.382675,29.738908


In [ ]:
crime_2024.dtypes

,0
Incident,int64
RMSOccurrenceDate,datetime64[ns]
RMSOccurrenceHour,int64
NIBRSClass,object
NIBRSDescription,object
OffenseCount,int64
Beat,object
Premise,object
StreetNo,object
StreetName,object


### **Remove unecessary columns**

In [ ]:
cols_to_drop = ['Incident', 'RMSOccurrenceHour', 'NIBRSClass', 'Beat', 'Suffix']
crime_2024 = crime_2024.drop(columns=cols_to_drop)

### **Rename columns**

In [ ]:
crime_2024.rename(columns={'RMSOccurrenceDate':'occurrence_date','NIBRSDescription':'nibrs_description','OffenseCount':'offense_count','Premise':'premise','StreetNo':'street_no','StreetName':'street_name','StreetType':'street_type','City':'city','ZIPCode':'zipcode','MapLongitude':'longitude','MapLatitude':'latitude'},inplace=True)

### **Drop NA ZIP Codes and Convert to String**

In [ ]:
crime_2024 = crime_2024.dropna(subset=['zipcode'])
crime_2024['zipcode']=crime_2024['zipcode'].astype(int).astype(str)

### **Convert All String Columns to Lowercase**

In [ ]:
crime_2024 = crime_2024.apply(lambda col: col.str.lower() if col.dtype == "object" else col)

In [ ]:
crime_2024

,occurrence_date,nibrs_description,offense_count,premise,street_no,street_name,street_type,city,zipcode,longitude,latitude
0,2024-01-01,disorderly conduct,1,"residence, home (includes apartment)",8404,eastover,st,houston,77028,-95.271852,29.823953
1,2024-01-01,disorderly conduct,1,"residence, home (includes apartment)",7125,glass,st,houston,77016,-95.325797,29.824464
2,2024-01-01,aggravated assault,1,"residence, home (includes apartment)",329,innsdale,dr,houston,77076,-95.371408,29.850945
3,2024-01-01,weapon law violations,1,"residence, home (includes apartment)",3727,dalmatian,dr,houston,77045,-95.432224,29.618177
4,2024-01-01,aggravated assault,1,"parking lot, garage",230,alabama,st,houston,77006,-95.382675,29.738908
...,...,...,...,...,...,...,...,...,...,...,...
250163,2024-12-31,simple assault,1,"residence, home (includes apartment)",1514,bingle,rd,houston,77055,-95.500562,29.797544
250164,2024-12-31,disorderly conduct,1,"residence, home (includes apartment)",701,victoria,dr,houston,77022,-95.388540,29.833788
250165,2024-12-31,"drug, narcotic violations",1,"residence, home (includes apartment)",701,victoria,dr,houston,77022,-95.388540,29.833788
250166,2024-12-31,drug equipment violations,1,"residence, home (includes apartment)",701,victoria,dr,houston,77022,-95.388540,29.833788


In [ ]:
crime_2024.dtypes

,0
occurrence_date,datetime64[ns]
nibrs_description,object
offense_count,int64
premise,object
street_no,object
street_name,object
street_type,object
city,object
zipcode,object
longitude,float64


## **1.4 Crime Data 2025**

### **Remove unecessary columns**

In [ ]:
cols_to_drop = ['Incident', 'RMSOccurrenceHour', 'NIBRSClass', 'Beat', 'Suffix','month']
crime_2025 = crime_2025.drop(columns = cols_to_drop)

### **Rename columns**

In [ ]:
crime_2025.rename(columns={'RMSOccurrenceDate':'occurrence_date','NIBRSDescription':'nibrs_description','OffenseCount':'offense_count','Premise':'premise','StreetNo':'street_no','StreetName':'street_name','StreetType':'street_type','City':'city','ZIPCode':'zipcode','MapLongitude':'longitude','MapLatitude':'latitude'},inplace=True)

### **Drop NA ZIP Codes and Convert to String**

In [ ]:
crime_2025 = crime_2025.dropna(subset=['zipcode'])
crime_2025['zipcode'] = crime_2025['zipcode'].astype(int).astype(str)

### **Convert All String Columns to Lowercase**

In [ ]:
crime_2025  = crime_2025.apply(lambda col:  col.str.lower() if col.dtype == "object" else col)

In [ ]:
crime_2025

,occurrence_date,nibrs_description,offense_count,premise,street_no,street_name,street_type,city,zipcode,longitude,latitude
0,2025-01-01,aggravated assault,1,"residence, home (includes apartment)",NaN,sherwood,ln,houston,77092,-95.458005,29.814084
1,2025-01-01,"drug, narcotic violations",1,"residence, home (includes apartment)",NaN,sherwood,ln,houston,77092,-95.458005,29.814084
2,2025-01-01,aggravated assault,1,"residence, home (includes apartment)",NaN,telephone,rd,houston,77075,-95.285279,29.618563
3,2025-01-01,all other offenses,1,"residence, home (includes apartment)",NaN,sumpter,NaN,houston,77026,-95.338518,29.779765
4,2025-01-01,disorderly conduct,1,"parking lot, garage",NaN,crestdale,dr,houston,77080,-95.530849,29.821221
...,...,...,...,...,...,...,...,...,...,...,...
35039,2025-02-28,aggravated assault,2,arena/stadium/fairgrounds/coliseum,NaN,fannin,st,houston,77054,-95.403217,29.683226
35040,2025-02-28,all other offenses,1,arena/stadium/fairgrounds/coliseum,NaN,fannin,st,houston,77054,-95.403217,29.683226
35041,2025-02-28,theft from motor vehicle,1,"parking lot, garage",NaN,congress,st,houston,77002,-95.359659,29.761623
35042,2025-02-28,motor vehicle theft,1,"residence, home (includes apartment)",NaN,saddle rock,dr,houston,77037,-95.409179,29.891754


In [ ]:
crime_2025.dtypes

,0
occurrence_date,datetime64[ns]
nibrs_description,object
offense_count,int64
premise,object
street_no,object
street_name,object
street_type,object
city,object
zipcode,object
longitude,float64


## **Save df to csv files**

In [ ]:
property_listings.to_csv("property_listings.csv", index=False)
zhvi.to_csv("zhvi.csv", index=False)
zori.to_csv("zori.csv", index=False)
zori1.to_csv("zori1.csv", index=False)
crime_2024.to_csv("crime_2024.csv", index=False)
crime_2025.to_csv("crime_2025.csv", index=False)